# <span style="color:green"><center>Diplomado en Big Data</center></span>

# <span style="color:red"><center>Almacenamiento de dataframes<center></span>

<img src="../images/dask_horizontal.svg" align="right" width="30%">


##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Contenido</span>


* [Introducción](#Introducción)


## <span style="color:blue">Fuente</span>

Esta es una traducción libre del tutorial disponible en [dask-tutorial](https://github.com/dask/dask-tutorial).

## <span style="color:blue">Introducción</span>

<img src="../images/hdd.jpg" width="20%" align="right">


El almacenamiento eficiente puede mejorar drásticamente el rendimiento, especialmente cuando se opera repetidamente desde el disco.

Descomprimir texto y analizar archivos CSV es caro. Una de las estrategias más efectivas  es utilizar un formato de almacenamiento binario como HDF5. A menudo, las ganancias de rendimiento al hacer esto son suficientes para que pueda volver a usar Pandas nuevamente en lugar de usar `dask.dataframe`.

En esta sección, aprenderemos cómo organizar y almacenar de manera eficiente sus conjuntos de datos en formatos binarios en disco. Usaremos lo siguiente:


1.  Formato [Pandas `HDFStore`](http://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5) sobre `HDF5`
2.  `Categoricals` para almacenar datos de texto numéricamente

**Principales conclusiones**

1. Los formatos de almacenamiento afectan el rendimiento en un orden de magnitud
2. Los datos de texto mantendrán lento incluso un formato rápido como HDF5
3. Una combinación de formatos binarios, almacenamiento de columnas y datos particionados convierte tiempos de espera de un segundo en tiempos de espera de 80 ms.

### Crear datos

In [ ]:
%run prep.py -d accounts

## <span style="color:blue">Lectura CSV</span>

Primero leemos nuestros datos *csv* como antes.

CSV y otros formatos de archivo basados en texto son el almacenamiento más común para datos de muchas fuentes, ya que requieren un preprocesamiento mínimo, se pueden escribir línea por línea y son legibles por humanos. Dado que Pandas '`read_csv` está bien optimizado, los CSV son una entrada razonable, pero están lejos de estar optimizados, ya que la lectura requiere un análisis de texto extenso.

In [1]:
import os
filename = os.path.join('../data', 'accounts.*.csv')
filename

'../data/accounts.*.csv'

In [2]:
import dask.dataframe as dd
df_csv = dd.read_csv(filename)
df_csv.head()

,id,names,amount
0,66,Ursula,-839
1,272,Victor,422
2,275,Hannah,406
3,267,Victor,888
4,318,Sarah,795


### Escritura a  HDF5

HDF5 y netCDF son formatos de arreglos binarios muy utilizados en el ámbito científico.

Pandas contiene un formato HDF5 especializado, `HDFStore`. El método `dd.DataFrame.to_hdf` funciona exactamente como el método `pd.DataFrame.to_hdf`.

In [29]:
target = os.path.join('../data', 'accounts.h5')
target

'../data/accounts.h5'

In [30]:
# convert to binary format, takes some time up-front
%time df_csv.to_hdf(target, '../data')

CPU times: user 4.54 s, sys: 320 ms, total: 4.85 s
Wall time: 4.89 s


['../data/accounts.h5', '../data/accounts.h5', '../data/accounts.h5']

In [31]:
# same data as before
df_hdf = dd.read_hdf(target, '../data')
df_hdf.head()

,id,names,amount
0,66,Ursula,-839
1,272,Victor,422
2,275,Hannah,406
3,267,Victor,888
4,318,Sarah,795


### Comparación de las velocidades CSV y HDF5

Hacemos un cálculo simple que requiere leer una columna de nuestro conjunto de datos y comparar el rendimiento entre los archivos CSV y nuestro archivo HDF5 recién creado. ¿Cuál esperas que sea más rápido?

In [32]:
%time df_csv.amount.sum().compute()

CPU times: user 863 ms, sys: 102 ms, total: 964 ms
Wall time: 423 ms


2661358724

In [33]:
%time df_hdf.amount.sum().compute()

CPU times: user 3.1 s, sys: 338 ms, total: 3.44 s
Wall time: 3.4 s


2661358724

Lamentablemente, son casi iguales, o quizás incluso más lentos.

El culpable aquí es la columna `names`, que es de tipo`object` y, por lo tanto, es difícil de almacenar de manera eficiente. Hay dos problemas aquí:

1. ¿Cómo almacenamos datos de texto como *nombres* de manera eficiente en el disco?
2. ¿Por qué tuvimos que leer la columna *nombres* cuando todo lo que queríamos era *cantidad*?

###   Almacenamiento eficiente de carateres con categoricals

Podemos usar categorías de Pandas para reemplazar nuestros tipos de objetos con una representación numérica. Esto lleva un poco más de tiempo desde el principio, pero da como resultado un mejor rendimiento.

Más sobre categóricos en el [pandas docs](http://pandas.pydata.org/pandas-docs/stable/categorical.html) y en [este blogpost](http://matthewrocklin.com/blog/work/2015/06/18/Categoricals).

In [34]:
# Categorize data, then store in HDFStore
%time df_hdf.categorize(columns=['names']).to_hdf(target, '../data2')

CPU times: user 9.54 s, sys: 791 ms, total: 10.3 s
Wall time: 10.2 s


['../data/accounts.h5', '../data/accounts.h5', '../data/accounts.h5']

In [35]:
# It looks the same
df_hdf = dd.read_hdf(target, '../data2')
df_hdf.head()

,id,names,amount
0,66,Ursula,-839
1,272,Victor,422
2,275,Hannah,406
3,267,Victor,888
4,318,Sarah,795


In [36]:
# But loads more quickly
%time df_hdf.amount.sum().compute()

CPU times: user 316 ms, sys: 94.6 ms, total: 410 ms
Wall time: 410 ms


2661358724

Esto ahora es definitivamente más rápido que antes. Esto nos dice que no es solo el tipo de archivo que usamos, sino también cómo representamos nuestras variables lo que influye en el rendimiento del almacenamiento.

¿Cómo depende el rendimiento de la lectura del programador que utilizamos? Puede probar esto con subprocesos, procesos y distribuidos.

Sin embargo, esto aún puede ser mejor. Tuvimos que leer todas las columnas (`nombres` y `monto`) para calcular la suma de uno (`monto`). Mejoraremos aún más esto con `parquet`, un almacén de columnas en disco. Sin embargo, primero aprendemos cómo establecer un índice en un dask.dataframe.

### Ejercicio

`fastparquet` es una librería para interactuar con archivos en formato parquet, que son un formato muy común en el ecosistema de Big Data, y utilizado por herramientas como Hadoop, Spark e Impala.

In [37]:
target = os.path.join('../data', 'accounts.parquet')
df_csv.categorize(columns=['names']).to_parquet(target, storage_options={"has_nulls": True}, engine="fastparquet")

Investigue la estructura de archivos en el nuevo directorio resultante: ¿para qué supone que son esos archivos?

`to_parquet` viene con muchas opciones, como compresión, si escribir explícitamente información NULL (no es necesario en este caso) y cómo codificar cadenas. Puede experimentar con ellos para ver qué efecto tienen en el tamaño del archivo y los tiempos de procesamiento, a continuación.

In [38]:
ls -l data/accounts.parquet/

total 73268
-rw-rw-r-- 1 alvaro alvaro      952 Jun 16 16:55 _common_metadata
-rw-rw-r-- 1 alvaro alvaro     2227 Jun 16 16:55 _metadata
-rw-rw-r-- 1 alvaro alvaro 25002096 Jun 16 16:55 part.0.parquet
-rw-rw-r-- 1 alvaro alvaro 25002096 Jun 16 16:55 part.1.parquet
-rw-rw-r-- 1 alvaro alvaro 25002096 Jun 16 16:55 part.2.parquet


In [39]:
df_p = dd.read_parquet(target)
# note that column names shows the type of the values - we could
# choose to load as a categorical column or not.
df_p.dtypes

id           int64
names     category
amount       int64
dtype: object

Vuelva a ejecutar el cálculo de la suma anterior para esta versión de los datos y el tiempo que tarda. Es posible que desee probar esto más de una vez; es común que muchas bibliotecas realicen varios trabajos de configuración cuando se llaman por primera vez.

In [40]:
%time df_p.amount.sum().compute()

CPU times: user 76.4 ms, sys: 60.5 ms, total: 137 ms
Wall time: 76.6 ms


2661358724

Al archivar datos, es común ordenar y dividir por una columna con identificadores únicos, para facilitar búsquedas rápidas más adelante. Para estos datos, esa columna es "id". Mide cuánto tiempo lleva recuperar las filas correspondientes a `id == 100` del CSV sin procesar, de HDF5 y versiones de parquet, y finalmente de una nueva versión de parquet escrita después de aplicar `set_index('id')`.

In [ ]:
# df_p.set_index('id').to_parquet(...)

## <span style="color:blue">Archivos remotos</span>

Dask puede acceder a varios servicios de almacenamiento de datos orientados a la nube y al clúster, como Amazon S3 o HDFS

Ventajas:
* almacenamiento seguro y escalable

Desventajas:
* la velocidad de la red se convierte en un cuello de botella

La forma de configurar marcos de datos (y otras colecciones) sigue siendo muy similar a antes. Tenga en cuenta que los datos aquí están disponibles de forma anónima, pero en general se puede pasar un parámetro adicional `storage_options =` con más detalles sobre cómo interactuar con el almacenamiento remoto.

```python
taxi = dd.read_csv('s3://nyc-tlc/trip data/yellow_tripdata_2015-*.csv',
                   storage_options={'anon': True})
```

**Advertencia**: las operaciones a través de Internet pueden tardar mucho en ejecutarse. Estas operaciones funcionan muy bien en una configuración en clúster en la nube, por ejemplo, máquinas de Amazon EC2 que leen desde S3 o máquinas de cómputo de Google que leen desde GCS.